# Challenge DataIQ - Debugging

## Definir variables a utilizar

In [40]:
from dataclasses import dataclass

## Definicion de variables a utilizar
@dataclass
class Config:

    archivos=['Episodios_Diagnosticos.csv',     # Nombre de los archivos que contienen la información a procesar
              'Estudios_Complementarios.csv',
              'Pacientes.csv',
              'Signos_Vitales.csv']

    path = "./data/"    # Ruta donde se encuentran los archivos

    n_charenc=10000   # cantidad de caracteres para evaluar el encoding de los archivos

    features=['EDAD',                     # features a considerar para el modelo (chequear que los nombres correspondan a features en df_final)
              'SEXO', 
              'CANTIDAD_EPISODIOS', 
              'CANTIDAD_ESTUDIOS', 
              'CANTIDAD_SIGNOS_VITALES',
              'PRIMER_AREA_FRECUENTE',
              'ULTIMO_AREA_FRECUENTE',
              'TIPO_DIAGNOSTICO_FRECUENTE'] 
    
    test_size=0.20     # proporcion del dataset de testing
    seed=42            # semilla de randomizacion

    model="Random Forest"  # modelo a utilizar en el entranmiento y evaluacion 
    cv_split=10            # segmentacion para el proceso de cross-validation
    

In [42]:
config=Config()

## Preprocess Data

In [51]:
# Evaluacion, lectura y carga de datos

from src.preprocessing import load_and_read_data
dfs = load_and_read_data(file_names=config.archivos, path=config.path, n_charenc=config.n_charenc)

./data/Episodios_Diagnosticos.csv
./data/
Successfully read Episodios_Diagnosticos.csv with encoding: ISO-8859-1
./data/Estudios_Complementarios.csv
./data/
Successfully read Estudios_Complementarios.csv with encoding: ISO-8859-1
./data/Pacientes.csv
./data/
Successfully read Pacientes.csv with encoding: ISO-8859-1
./data/Signos_Vitales.csv
./data/
Successfully read Signos_Vitales.csv with encoding: ISO-8859-1
Total DataFrames loaded: 4


In [47]:
# Division de dataframes para cada archivo 

df_episodios = dfs[0]
df_estudios = dfs[1]
df_pacientes = dfs[2]
df_signos = dfs[3]

In [54]:
# Agregar 'CLASE' al df de Episodios en base a 'TIPO_EPISODIO' ('CLASE'=1 si 'TIPO_EPISODIO' = 'H',  'CLASE' = 0 en otro caso)

from src.preprocessing import crear_tipo_episodio
df_episodios = crear_tipo_episodio(df_episodios)

In [56]:
# Limpieza inicial, especificada en la documentación: Eliminacion de registros con 'TIPO_EPISODIO'='*' en df_episodios, 
#                                                     Eliminación de registros ducplicados en df_pacientes,
#                                                     Eliminacion de columno 'ID_ITEM' en df_estudios. 

from src.preprocessing import default_clean
df_episodios, df_pacientes, df_estudios = default_clean(df_episodios, df_pacientes, df_estudios)

In [58]:
# Creacion del DataFrame final, que será el dataset que se dividirá en entrenamiento y testing. Consultar documentacion aparte sobre que criterios se consideraron. 
# 'create_final_dataframe()' en 'preprocessing.py' contiene ena explicacion parcial de la misma

from src.preprocessing import create_final_dataframe
df_final = create_final_dataframe(df_pacientes, df_episodios, df_signos, df_estudios)

In [59]:
# Revisión de la longitud del dataset. Dado que los registros que se utilizaran corresponden a pacientes,
#  el df_final debe ser de la misma longitud que df_pacientes, ya que la informacion concatenada en el mismo es relativa a los pacientes. 

from src.preprocessing import check_final_data
check_final_data(df_final,df_pacientes)

La cantidad de registros es la correcta: total de  2635


In [60]:
# Extraccion de los conjuntos de entrenamiento y testeo. Se seleccionan los features, la proporcion del set de test y
# la semilla de randomizacion (todas en el archivo de configuracion)

from src.preprocessing import prepare_test_train
X_train, X_test, y_train, y_test = prepare_test_train(df_final,features=config.features, test_size=config.test_size, seed=config.seed)

# Entrenamiento y evaluacion del modelo

In [65]:
# Entrenamiento y evaluacion en una serie de modelos posibles. Eleccion del mejor modelo (en base unicamente a la precision)

from src.modeling import evaluate_models
df_evaluate_models, best_model = evaluate_models(X_train, X_test, y_train, y_test, test_size=config.test_size, random_state=config.seed)

                    Model  Accuracy  Precision    Recall  F1-Score       AUC
1           Random Forest  0.912713   0.900552  0.853403  0.876344  0.962766
6       Gradient Boosting  0.912713   0.896175  0.858639  0.877005  0.960865
0     Logistic Regression  0.910816   0.900000  0.848168  0.873315  0.951306
3           Decision Tree  0.895636   0.857895  0.853403  0.855643  0.886523
5             Naive Bayes  0.886148   0.832487  0.858639  0.845361  0.935007
4     K-Nearest Neighbors  0.850095   0.804348  0.774869  0.789333  0.885845
2  Support Vector Machine  0.842505   0.772727  0.801047  0.786632  0.884271
Best performance model in Accuracy is:  Random Forest


In [67]:
# Agrego el mejor modelo al archivo de configuracion

config.model = best_model
print(config.model)

Random Forest


In [70]:
# Dado el mejor modelo en la evaluacion, re-entreno el mismo y lo re-evaluo

from src.modeling import train_and_evaluate_model 
model, accuracy, precission, recall, f1, auc = train_and_evaluate_model(X_train, X_test, y_train, y_test, model=config.model, seed=config.seed)
#model, accuracy=train_and_evaluate_model(X_train, X_test, y_train, y_test, model="Gradient Boosting", seed=config.seed)

Modelo utilizado:  Random Forest
Exactitud del modelo: 0.91


OBS: Se podria haber obtenido directamente el objeto modelo a partir de 'evaluate_models()', 
pero he optado por realizar ésta segunda accion por separado en base al siguiente criterio:

1) En esta instancia no es relevante optimizar la eficiencia
2) El tamaño del dataset, y por lo tanto tambien el tiempo de entrenamiento de los modelos evaluados, no significan un consumo elevado de recursos en la instancia actual de devlopment (sí tendría un impacto en una posible instancia de produccion). 
3) En pos de la claridad del codigo en esta instancia resulta preferible no anidar demasiadas funciones, por lo tanto prefiero no reutilizar 'train_and_evaluate_model()' dentro de 'evaluate_models()'
   

In [73]:
# Evaluacion de metricas tipicas para el modelo entrenado: cross-validation, matriz de confusion, recall, F1

from src.evaluation import full_metrics_eval
cv_score, y_pred, cm, cr=full_metrics_eval(model, X_train, y_train, X_test, y_test, cv_split=config.cv_split)

Promedio de los puntajes de validación cruzada: 0.92
Matriz de Confusión:
[[318  18]
 [ 28 163]]
Matriz de confusión porcentual:
[[94.64  5.36]
 [14.66 85.34]]
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.92      0.95      0.93       336
           1       0.90      0.85      0.88       191

    accuracy                           0.91       527
   macro avg       0.91      0.90      0.90       527
weighted avg       0.91      0.91      0.91       527



In [75]:
features_alt_1 = ['EDAD',                     
                  'SEXO', 
                  #'CANTIDAD_EPISODIOS', 
                  #'CANTIDAD_ESTUDIOS', 
                  #'CANTIDAD_SIGNOS_VITALES',
                  'PRIMER_AREA_FRECUENTE',
                  'ULTIMO_AREA_FRECUENTE',
                  'TIPO_DIAGNOSTICO_FRECUENTE'
                  ] 

features_alt_2 = ['EDAD',                     
                  'SEXO', 
                  #'CANTIDAD_EPISODIOS', 
                  #'CANTIDAD_ESTUDIOS', 
                  #'CANTIDAD_SIGNOS_VITALES',
                  #'PRIMER_AREA_FRECUENTE',
                  'ULTIMO_AREA_FRECUENTE',
                  'TIPO_DIAGNOSTICO_FRECUENTE'
                  ]

features_alt_3 = ['EDAD',                     
                  'SEXO', 
                  'CANTIDAD_EPISODIOS', 
                  'CANTIDAD_ESTUDIOS', 
                  'CANTIDAD_SIGNOS_VITALES',
                  'PRIMER_AREA_FRECUENTE',
                  'ULTIMO_AREA_FRECUENTE',
                  'TIPO_DIAGNOSTICO_FRECUENTE'
                  ]


In [78]:
from src.preprocessing import prepare_test_train
X_train, X_test, y_train, y_test = prepare_test_train(df_final,features=features_alt_1, test_size=config.test_size, seed=config.seed)
df_evaluate_models, best_model = evaluate_models(X_train, X_test, y_train, y_test, test_size=config.test_size, random_state=config.seed)
config.model = best_model
model, accuracy, precission, recall, f1, auc = train_and_evaluate_model(X_train, X_test, y_train, y_test, model=config.model, seed=config.seed)
cv_score, y_pred, cm, cr=full_metrics_eval(model, X_train, y_train, X_test, y_test, cv_split=config.cv_split)

                    Model  Accuracy  Precision    Recall  F1-Score       AUC
0     Logistic Regression  0.905123   0.907514  0.821990  0.862637  0.936986
6       Gradient Boosting  0.891841   0.885057  0.806283  0.843836  0.940718
3           Decision Tree  0.889943   0.863388  0.827225  0.844920  0.886507
1           Random Forest  0.886148   0.846561  0.837696  0.842105  0.936986
5             Naive Bayes  0.884250   0.831633  0.853403  0.842377  0.931626
4     K-Nearest Neighbors  0.867173   0.834254  0.790576  0.811828  0.902214
2  Support Vector Machine  0.815939   0.709821  0.832461  0.766265  0.883009
Best performance model in Accuracy is:  Logistic Regression
Modelo utilizado:  Logistic Regression
Exactitud del modelo: 0.90
Promedio de los puntajes de validación cruzada: 0.91
Matriz de Confusión:
[[320  16]
 [ 36 155]]
Matriz de confusión porcentual:
[[95.24  4.76]
 [18.85 81.15]]
Reporte de Clasificación:
              precision    recall  f1-score   support

           0     

In [79]:
X_train, X_test, y_train, y_test = prepare_test_train(df_final,features=features_alt_2, test_size=config.test_size, seed=config.seed)
df_evaluate_models, best_model = evaluate_models(X_train, X_test, y_train, y_test, test_size=config.test_size, random_state=config.seed)
config.model = best_model
model, accuracy, precission, recall, f1, auc = train_and_evaluate_model(X_train, X_test, y_train, y_test, model=config.model, seed=config.seed)
cv_score, y_pred, cm, cr=full_metrics_eval(model, X_train, y_train, X_test, y_test, cv_split=config.cv_split)

                    Model  Accuracy  Precision    Recall  F1-Score       AUC
0     Logistic Regression  0.905123   0.907514  0.821990  0.862637  0.938061
5             Naive Bayes  0.901328   0.897143  0.821990  0.857923  0.927574
3           Decision Tree  0.897533   0.878453  0.832461  0.854839  0.897454
6       Gradient Boosting  0.895636   0.890805  0.811518  0.849315  0.943211
1           Random Forest  0.893738   0.857143  0.848168  0.852632  0.937407
4     K-Nearest Neighbors  0.865275   0.848837  0.764398  0.804408  0.892569
2  Support Vector Machine  0.814042   0.706667  0.832461  0.764423  0.871284
Best performance model in Accuracy is:  Logistic Regression
Modelo utilizado:  Logistic Regression
Exactitud del modelo: 0.90
Promedio de los puntajes de validación cruzada: 0.91
Matriz de Confusión:
[[321  15]
 [ 37 154]]
Matriz de confusión porcentual:
[[95.54  4.46]
 [19.37 80.63]]
Reporte de Clasificación:
              precision    recall  f1-score   support

           0     

In [80]:
X_train, X_test, y_train, y_test = prepare_test_train(df_final,features=features_alt_3, test_size=config.test_size, seed=config.seed)
df_evaluate_models, best_model = evaluate_models(X_train, X_test, y_train, y_test, test_size=config.test_size, random_state=config.seed)
config.model = best_model
model, accuracy, precission, recall, f1, auc = train_and_evaluate_model(X_train, X_test, y_train, y_test, model=config.model, seed=config.seed)
cv_score, y_pred, cm, cr=full_metrics_eval(model, X_train, y_train, X_test, y_test, cv_split=config.cv_split)

                    Model  Accuracy  Precision    Recall  F1-Score       AUC
1           Random Forest  0.912713   0.900552  0.853403  0.876344  0.962766
6       Gradient Boosting  0.912713   0.896175  0.858639  0.877005  0.960865
0     Logistic Regression  0.910816   0.900000  0.848168  0.873315  0.951306
3           Decision Tree  0.895636   0.857895  0.853403  0.855643  0.886523
5             Naive Bayes  0.886148   0.832487  0.858639  0.845361  0.935007
4     K-Nearest Neighbors  0.850095   0.804348  0.774869  0.789333  0.885845
2  Support Vector Machine  0.842505   0.772727  0.801047  0.786632  0.884271
Best performance model in Accuracy is:  Random Forest
Modelo utilizado:  Random Forest
Exactitud del modelo: 0.91
Promedio de los puntajes de validación cruzada: 0.92
Matriz de Confusión:
[[318  18]
 [ 28 163]]
Matriz de confusión porcentual:
[[94.64  5.36]
 [14.66 85.34]]
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.92      

In [86]:
from src.evaluation import save_model_with_metadata

save_model_with_metadata(model, 
                         model_name, 
                         metrics, 
                         cv_score, 
                         classification_report, 
                         config, 
                         model_dir="models", 
                         log_file="model_log.csv")

NameError: name 'model_name' is not defined